In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
from geopy.distance import great_circle
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

import time

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server
name_project

'project_repartition_v3.0/output_repar_v9.1_02-02/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_v9.1_02-02/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--07-03',
 '2--08-03',
 '3--09-03',
 '4--10-03',
 '5--11-03',
 '6--12-03',
 '7--13-03',
 '8--14-03',
 '9--15-03',
 '10--16-03',
 '11--18-03',
 '12--19-03',
 '13--20-03',
 '14--21-03',
 '15--22-03',
 '16--23-03',
 '17--24-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '24--32-03',
 '25--33-03',
 '26--34-03',
 '27--36-03',
 '28--37-03',
 '29--38-03',
 '30--39-03',
 '31--40-03',
 '32--41-03',
 '33--42-03',
 '34--43-03',
 '35--44-03',
 '36--45-03',
 '37--46-03',
 '38--47-03',
 '39--48-03',
 '40--49-03',
 '41--50-03',
 '42--51-03',
 '43--52-03',
 '44--53-03',
 '45--54-03',
 '46--07-04',
 '47--08-04',
 '48--09-04',
 '49--10-04',
 '50--11-04',
 '51--12-04',
 '52--13-04',
 '53--14-04',
 '54--16-04',
 '55--17-04',
 '56--18-04',
 '57--19-04',
 '58--20-04',
 '59--21-04',
 '60--22-04',
 '61--23-04',
 '62--24-04',
 '63--25-04',
 '64--26-04',
 '65--27-04',
 '66--28-04',
 '67--29-04',
 '68--30-04',
 '69--31-04',
 '70--32-04',
 '71--33-04',
 '72--34-04',
 

In [7]:
if 'key_subworks_repick' in info_basic.keys():
    key_subworks_repick = info_basic['key_subworks_repick']
else:
    key_subworks_repick = []
key_subworks_repick

[]

In [8]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic['name_CC']
dir_partition = dir_project + info_basic['dir_partition']
#dir_CC = dir_CC_workspace+'CC/CC_40_prewhiten/'

In [9]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [10]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [11]:
info_basic_bi['r_max'] = {}

In [12]:
def linear_stack(key_subwork):
    global key_subworks
    global dir_stack
    global info_basic
    global stainfo_all
    global dir_partition
    global key_subworks_repick

    print("Collecting ",key_subwork,' ...')
    outname = str(key_subwork)+'_gather_linear.h5'
    if key_subwork in key_subworks_repick:
        if os.path.exists(dir_stack+outname):
            os.remove(dir_stack+outname)
    if os.path.exists(dir_stack+str(key_subwork)+'_gather_linear.h5'):
        print("File exists.")
        return

    time0 = time.time()

    nf = info_basic['nf']
    
    filepath = dir_partition + str(key_subwork) + '.txt'
    stalist, lat, lon = np.loadtxt(filepath, dtype='str', unpack=True)
    nsta = len(stalist)
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = np.min( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        idx2 = np.max( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        #idx1 = int(stalist_all.index(stalist[sta1]))
        #idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m +idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        count[i] = count_all[num]
        #r[i] = r0[num]
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])

    
    ncffile = h5py.File(dir_stack+outname,'w')
    ncffile.create_dataset('ncfs',data=ncfs_sum_linear)
    ncffile.create_dataset('r',data=r)
    ncffile.create_dataset('count',data=count)
    ncffile.create_dataset('f',data=f)
    ncffile.create_dataset('StationPairs',data=StationPairs)
    #print(ncffile.keys())
    ncffile.close()
    #np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.", str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))

In [13]:
nThreads = 60

In [14]:
for key_subwork in key_subworks:
    linear_stack(key_subwork)

Done in  0.3736090660095215  s. 1 / 716
Done in  0.37715983390808105  s. 2 / 716
Done in  0.5187149047851562  s. 3 / 716
Done in  0.5800325870513916  s. 4 / 716
Done in  0.6582911014556885  s. 5 / 716
Done in  0.767371416091919  s. 6 / 716
Done in  0.7142820358276367  s. 7 / 716
Done in  1.121417760848999  s. 8 / 716
Done in  0.832646369934082  s. 9 / 716
Done in  0.9165139198303223  s. 10 / 716
Done in  1.0843684673309326  s. 11 / 716
Done in  1.2731921672821045  s. 12 / 716
Done in  1.399224042892456  s. 13 / 716
Done in  1.0143475532531738  s. 14 / 716
Done in  0.6591770648956299  s. 15 / 716
Done in  0.3749215602874756  s. 16 / 716
Done in  0.4553396701812744  s. 17 / 716
Done in  0.945559024810791  s. 18 / 716
Done in  0.6700859069824219  s. 19 / 716
Done in  0.776355504989624  s. 20 / 716
Done in  0.9036900997161865  s. 21 / 716
Done in  1.858048439025879  s. 22 / 716
Done in  1.182387351989746  s. 23 / 716
Done in  1.1624319553375244  s. 24 / 716
Done in  0.4068310260772705  s. 

In [15]:
np.save(filename_bi,info_basic_bi)